#              Исследование надёжности заёмщиков



Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных: 

- children - количество детей в семье
- days_employed - общий трудовой стаж в днях
- dob_years - возраст клиента в годах
- education - уровень образования клиента
- education_id - идентификатор уровня образования
- family_status - семейный статус
- family_status_id - идентификатор семейного статуса
- gender - пол клиента 
- income_type - тип занятости
- debt - имел ли задолженность по возврату кредитов 
- total_income - ежемесячный доход
- purpose - цель получения кредита

### Шаг 1. Изучаем общую информацию о данных

In [1]:
import pandas as pd
data = pd.read_csv('C:\\data.csv')
data.shape

(21525, 12)

In [2]:
print(data.info())
print()
display(data.describe(include = 'all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


In [3]:
print(data[data['total_income'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


С помощью метода isnull() обнаружены 2174 пропуска(NaN) в столбцах: days_employed и total_income. 
Это около 10% от общего числа. **Такое количество пропусков может быть обусловлено нежеланием людей предоставлять банку данные о доходах и свой трудовой стаж.**

In [4]:
data['education'] = data['education'].apply(lambda x: str.lower(x)) #переводим весь текст в нижний регистр
data['days_employed'] = abs(data['days_employed']) / 365 #убираем отриц. значения и переводим дни в года
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
#опредеяем медианные значения дохода в соответствии с типом занятости
income_type_median = data.groupby('income_type')['total_income'].median()
print(income_type_median)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [6]:
#избавляемся от "-" значений в столбце 'стаж работы' и определяем медианные значения стажа работы в соответствии с возрастом клиента
data['days_employed'] = abs(data['days_employed'])
employed_median = data.groupby('dob_years')['days_employed'].median()

In [7]:
#функция принимает в качестве аргумента датафрем и тип занятости, и заменяет пустые значения в столбце 'total_income' на медианные по типу занятости
def fill_by_median(data, job_type):
    data.loc[data['income_type'] == job_type, 'total_income'] = data.loc[data['income_type'] == job_type, 'total_income'].fillna(income_type_median[job_type])

#применяем функцию    
fill_by_median(data,'госслужащий')
fill_by_median(data,'компаньон')
fill_by_median(data,'пенсионер')
fill_by_median(data,'предприниматель')
fill_by_median(data,'сотрудник')

In [8]:
#пишем цикл(чтобы не использовать функцию т.к у нас 58 'возрастов'),заменяющий пустые значения в стаже работы на медианные в соответствии с возрастом клиента
for age in data['dob_years']:
    data.loc[data['dob_years'] == age, 'days_employed'] = data.loc[data['dob_years'] == age, 'days_employed'].fillna(employed_median[age])

print(data.info()) # убедимся, что все NaN заполнены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Для того чтобы заполнить 2174 пропуска, мы применяем к 'days_employed' и 'total_income' цикл и функцию, которые заменяют все NaN на медианные значения по соответствующим параметрам: dob_years и income_type. Получается, что доход зависит от типа занятости клиента, а стаж - от возраста.

### Замена типа данных

In [9]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [10]:
#проверим данные по кол-во детей на корректность
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [11]:
# заменяем некорректные значения в столбце кол-во детей
data.loc[data['children']==20, 'children'] = 2
data.loc[data['children']==-1, 'children'] = 1
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Итог

Заменили вещественный тип данных на целочисленный, т.к стаж может быть только целым числом. Далее, а также с помощью логической индексации loc заменяем ошибочные значения кол-ва детей '20' и '-1' на верные.

### Обработка дубликатов

In [12]:
data = data.drop_duplicates()
print(data.duplicated().sum())#проверка, остались ли дубликаты

0


### Вывод

В нашем фрейме мы методом duplicated().sum() обнаружили 54 дубликата и методом drop_duplicates() удалили их. Возможно эти данные были собраны из нескольких таблиц и при объединении возникли дубликаты. Дубликаты вызывают искажения финальных результатов, поэтому их и надо удалять.

### Лемматизация

In [ ]:
# Отобразим все имеющиеся варианты цели кредита и определим, какие подстроки помогут нам правильно определить категорию
print(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [14]:
def category(row):
    if 'автомобиля' in row or 'автомобиль' in row or 'автомобилем' in row or 'автомобили' in row:
        return 'операции с автомобилем'
    if 'свадьба' in row or 'свадьбы' in row or 'свадьбу' in row:
        return 'проведение свадьбы'
    if 'образование' in row or 'образованием' in row or 'образования' in row or 'образованием' in row:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'
    
data['purpose_category'] = data['purpose'].apply(category)
print(data['purpose_category'].value_counts()) 

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64


### Вывод

Судя по полученным данным первое место по количеству заявок занимают кредиты на покупку и операции с недвижимостью. Это логично, т.к недвижимость самый востребованный и дорогой сегмент рынка.

### Категоризация данных

In [15]:
#делим всех клиентов банка на 5 квантилей, которые используем в функции ниже, как границы разделения группы по достатку
kvant = pd.qcut(data['total_income'], 5) 
print(kvant.value_counts())

(20666.999, 98537.6]     4291
(98537.6, 132134.4]      4291
(161335.0, 214618.2]     4291
(214618.2, 2265604.0]    4291
(132134.4, 161335.0]     4290
Name: total_income, dtype: int64


In [16]:
def income_category(x):
    
    if x <= 98538:    
        return 'very_low_inc'
    if 98538 < x <= 132134:
        return 'low_inc'
    if 132134 < x <= 161335:
        return 'medium_inc'
    if 161335 < x <= 214618:
        return 'high_inc'
    else:
        return 'very_high_inc'
    
data['income_categories'] = data['total_income'].apply(income_category)
print(data['income_categories'].value_counts())

very_high_inc    4291
low_inc          4291
very_low_inc     4291
high_inc         4291
medium_inc       4290
Name: income_categories, dtype: int64


### Вывод

Применив функцию qcut(), мы получаем распределение всех клиентов на 5 групп с одинаковым количеством наблюдений и на основе этих данных создаем категории доходности- very_low_inc, very_high_inc, high_inc, low_inc, medium_inc. В каждой группе содержится достаточное число клиентов для построения по ним выводов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
print()
print('% должников по количеству детей среди каждой группы заемщиков банка')
print()
print(data.groupby('children')['debt'].mean()*100)


% должников по количеству детей среди каждой группы заемщиков банка

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


### Вывод

Среди  должников банка меньше всего бездетных(7.54%), на втором месте люди с тремя детьми(8.18%), далее - с одним ребенком(9.16%), с двумя(9.49%) и четырьмя детьми(9.75%).Судя по тенденции(кроме должников с 3 детьми) можно сказать, что чем больше детей, тем реже люди возвращают кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
print()
print('% должников по семейному положению среди каждой группы заемщиков банка')
print()
print(data.groupby('family_status')['debt'].mean()*100)


% должников по семейному положению среди каждой группы заемщиков банка

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64


### Вывод

Среди  должников банка меньше всего вдовцов/вдов(6.56%), на втором месте люди в разводе(7.11%), далее - в браке(7.54%), в гражданском браке(9.34%) и не женат / не замужем(9.75%). Можно сказать, что самые безответственные группы- это люди  никогда не создававшие семьи или живущие в гражанском браке. Исходя из этого мы приходим к выводу, что семейное положение влияет на выплату кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
print()
print('% должников по уровню дохода среди каждой группы заемщиков банка')
print()
print(data.groupby('income_categories')['debt'].mean()*100)


% должников по уровню дохода среди каждой группы заемщиков банка

income_categories
high_inc         8.412957
low_inc          8.412957
medium_inc       8.741259
very_high_inc    6.991377
very_low_inc     8.016779
Name: debt, dtype: float64


### Вывод

Среди всех групп людей распределенных по доходам, примерно у всех одинаковое процентое распределение(примерно 8%), кроме группы людей с очень высоким доходом(6.99%).Отсюда делаем вывод, что уровень дохода практически не влияет на выплату кредита в срок, за исключением группы людей с очень высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
print()
print('% должников по целям кредита среди каждой группы заемщиков банка')
print()
print(pd.pivot_table(data, values=['debt'], index =['purpose_category'], aggfunc='mean')*100)


% должников по целям кредита среди каждой группы заемщиков банка

                              debt
purpose_category                  
операции с автомобилем    9.359034
операции с недвижимостью  7.233373
получение образования     9.220035
проведение свадьбы        8.003442


### Вывод

Среди  должников банка наиболее ответственная группа людей с целью кредита- недвижимость(7.23%), на втором месте группа людей с целью кредита- свадьба (8.00%), далее - образование(9.22%) и последняя группа людей с целью кредита- автомобиль(9.35%). Исходя из этого мы приходим к выводу, что цель кредита влияет на выплату кредита в срок и наиболее безответственная группа людей с целью кредита- автомобиль.

In [22]:
# Пример 2
print(data.groupby('children')['debt'].mean()*100)

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


### Шаг 4. Общий вывод

В самом начале работы, мы получили фрейм данных с 21525 строк.В ходе работы, нами было выявленно 2174 пропуска (NaN), которые были заменены на медианные по соответствующим параметрам. Так-же были выявленны и удалены  54 дубликата и заменены ошибочные значения кол-ва детей '20' и '-1' на верные- '2' и '1'. С помощью лемматизации мы привели строку 'цель кредита' к единообразию, т.е разбили строку на 4 четких категории- 'недвижимость', 'автомобиль', 'образование' и 'свадьба'. Использовав квантили, мы разделили всех клиентов по уровню дохода на 5 групп с одинаковым количеством наблюдений и на основе этих данных создали категории доходности- very_low_inc, very_high_inc, high_inc, low_inc, medium_inc. Обработав и проанализировав эти данные, мы приходим к выводу, что наличие детей(чем больше детей, тем хуже идут выплаты по кредиту), семейное положение(наиболее безответственные группы- это люди  никогда не создававшие семьи или живущие в гражанском браке), уровень ежемесячного дохода(люди с очень высоким доходом наиболее ответственные) и цель кредита(покупающие авто хуже всего производят выплаты по кредиту) влияют на своевременную выплату по кредиту.
Условный портрет наименее выгодного для банка заемщика - 4 ребенка, не женат(не замужем), со средним уровнем дохода и с целью кредита- автомобиль.
Условный портрет наиболее выгодного для банка заемщика - бездетный, вдовец(вдова), с очень высоким уровнем дохода и с целью кредита- недвижимость.